### Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [22]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [23]:
!kaggle competitions download -c fake-news

 58% 27.0M/46.5M [00:00<00:00, 132MB/s] 
100% 46.5M/46.5M [00:00<00:00, 162MB/s]


In [24]:
import zipfile
zip_ref = zipfile.ZipFile('fake-news.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [25]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

In [26]:
train=pd.read_csv("/content/test.csv")
test=pd.read_csv("/content/test.csv")

In [27]:
train.shape

(5200, 4)

In [28]:
test.shape

(5200, 4)

In [29]:
train.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [30]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


### Train and Test data is same

In [31]:
df=pd.read_csv("/content/train.csv")

In [32]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [33]:
# Droping the nan values
df.dropna(inplace=True)

In [34]:
df.shape

(18285, 5)

In [35]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [36]:
X=df.drop(columns="label")
y=df["label"]

In [37]:
X.shape, y.shape

((18285, 4), (18285,))

In [38]:
import tensorflow as tf

In [39]:
tf.__version__

'2.9.2'

In [67]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [68]:
# Vocublary size or Dictionary
voc_size=5000

In [69]:
# One Hot Representation
message=X.copy()

In [70]:
message=message.reset_index(drop=True)

In [71]:
message.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [72]:
message["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [73]:
import nltk
import re
from nltk.corpus import stopwords

In [74]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [75]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [76]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [77]:
one_hot_represent=[one_hot(words,voc_size) for words in corpus]
one_hot_represent

[[1421, 2486, 515, 3960, 414, 4435, 4331, 581, 1258, 2259],
 [4827, 2708, 3782, 1129, 3135, 2474, 4576],
 [2130, 4014, 316, 884],
 [2884, 1966, 2458, 1418, 2314, 3122],
 [3981, 3135, 1073, 1861, 3969, 2165, 3135, 3563, 3327, 2634],
 [3070,
  3091,
  4777,
  4985,
  451,
  491,
  376,
  4143,
  1688,
  3918,
  1607,
  4441,
  2165,
  4917,
  4576],
 [1695, 2493, 4774, 3814, 3152, 699, 4228, 4478, 1337, 3039, 1600],
 [4099, 658, 2669, 4819, 107, 2990, 491, 4231, 1337, 3039, 1600],
 [4018, 432, 4198, 2567, 796, 2360, 1035, 2044, 491, 2163],
 [4258, 1736, 4987, 3644, 12, 3133, 1866, 3510],
 [3745, 4981, 2405, 810, 4367, 1086, 2681, 2282, 1351, 2929, 3834],
 [1418, 3957, 414, 2360, 491, 107],
 [3484, 4566, 3568, 3563, 197, 3109, 48, 4799, 2460],
 [2485, 4219, 3898, 3340, 159, 3054, 1159, 1337, 3039, 1600],
 [1619, 594, 1118, 68, 4932, 1337, 3039, 1600],
 [3170, 2283, 1424, 4889, 614, 519, 2484, 1749, 4150, 4129],
 [1045, 1965, 2708],
 [3050, 3336, 979, 707, 491, 2280, 2248, 4576],
 [3254, 9

### Embedding Repersentaion

In [78]:
sent_length=20
embedded_docs=pad_sequences(one_hot_represent,padding="pre",maxlen=sent_length)

In [79]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1421,
       2486,  515, 3960,  414, 4435, 4331,  581, 1258, 2259], dtype=int32)

In [80]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [81]:
## Creating model BIDirectional
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [82]:
len(embedded_docs),y.shape

(18285, (18285,))

In [83]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [84]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=100)

### Model Training

In [86]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 13ms/step - loss: 0.3017 - accuracy: 0.8569 - val_loss: 0.2002 - val_accuracy: 0.9160
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1379 - accuracy: 0.9487 - val_loss: 0.2339 - val_accuracy: 0.9138
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0940 - accuracy: 0.9661 - val_loss: 0.2399 - val_accuracy: 0.9147
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0690 - accuracy: 0.9749 - val_loss: 0.2995 - val_accuracy: 0.9143
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0450 - accuracy: 0.9849 - val_loss: 0.3087 - val_accuracy: 0.9180
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0310 - accuracy: 0.9913 - val_loss: 0.4049 - val_accuracy: 0.9080
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0254 - accuracy: 0.9926 - val_loss: 0.3704 - val_accuracy: 0.9052

### Adding a Dropout layer

In [87]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrices

In [88]:
y_pred=(model.predict(X_test)> 0.5).astype("int32")

189/189 [==============================] - 1s 2ms/step


In [89]:
# y_pred=np.round(y_pred)

In [90]:
y_pred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [91]:
y_test

array([0, 1, 0, ..., 0, 0, 0])

In [92]:
from sklearn.metrics import confusion_matrix

In [93]:
confusion_matrix(y_test,y_pred)

array([[ 901, 2559],
       [ 651, 1924]])

In [94]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.46810273405136704